<a href="https://colab.research.google.com/github/dolmani38/similarly/blob/master/BERT_Sentence_similarity_0720.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jul 20 11:59:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import pandas as pd
# 한국어 영화 리뷰를 Load.
ds = pd.read_csv("https://raw.githubusercontent.com/dolmani38/similarly/master/ratings_test.txt",sep='\t')
ds

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


##Sentence Transformers: Multilingual Sentence Embeddings using BERT / RoBERTa / XLM-RoBERTa & Co. with PyTorch

참조:https://github.com/UKPLab/sentence-transformers

In [5]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 3.9MB/s 
     |████████████████████████████████| 778kB 10.8MB/s 
     |████████████████████████████████| 3.0MB 18.5MB/s 
     |████████████████████████████████| 890kB 29.3MB/s 
     |████████████████████████████████| 1.1MB 38.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-cp36-none-any.whl size=86756 sha256=d633ef0afd396bd1d74d1162fa7f89a324573221e66518faa874dba1175d7545
  Stored in directory: /root/.cache/pip/wheels/b8/23/85/85d6a9a6c68f0625a1ecdaad903bb0a78df058c10cf74f9de4
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1cc253435c51182924f1932549abb2eff41c739a0561112b0530672552b41477
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


##영어 + 한국어 BERT 모델의 사용

#Extending Sentence Embeddings Models to New Languages

#Available Pre-trained Models


참조:https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md

In [7]:
from sentence_transformers import SentenceTransformer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

100%|██████████| 1.80G/1.80G [00:34<00:00, 51.7MB/s]


In [8]:
#영화 리뷰의 내용을 corpus array에 저장한다.

corpus = []
for row in ds.iterrows():
    doc_id = row[1][0]
    doc_cont = str(row[1][1])
    corpus.append(doc_cont)

print('corpus size:',len(corpus))

corpus size: 50000


In [13]:
#하나의 sentence를 embedding해 본다.
#약 123ms 소요되며, 1024 차원의 vector를 생성한다.

from timeit import default_timer as timer
from datetime import timedelta
start = timer()

sentence_embedding = embedder.encode([corpus[4]],show_progress_bar=True)   

end = timer()

print("End... elapsed time : {}".format(timedelta(seconds=end-start)))
print('vector dim:',len(sentence_embedding[0]))
sentence_embedding


End... elapsed time : 0:00:00.126776
vector dim: 1024


[array([ 0.6906828 , -0.2728697 ,  0.541603  , ..., -0.58096015,
         0.6411979 , -0.16210863], dtype=float32)]

In [16]:
corpus[4]

'3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??'

In [18]:
# 50000개 전체 corpus를 embedding한다.

start = timer()
corpus_embeddings = embedder.encode(corpus[0:1000],show_progress_bar=True) 
end = timer()

print("End... elapsed time : {}".format(timedelta(seconds=end-start)))


End... elapsed time : 0:00:26.922158


In [21]:
# 유사한 sentence를 찾는 함수의 정의
import scipy

def searchSentence(queries,Threshold_score=0.60, Topn=0):
    index = {}
    n = len(queries)
    query_embeddings = embedder.encode(queries)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        
        for idx, distance in results[0:len(results) if Topn==0 else Topn]:
          score = 1-distance
          if not idx in index:
            index[idx] = 0.0
          index[idx] += float(score/n)

    return index

In [25]:
idx = searchSentence(['별 다섯 개'], Topn=20)
idx

{4: 0.3280455932703834,
 16: 0.3086543896496401,
 26: 0.3906515406231682,
 116: 0.3004176735042725,
 254: 0.3398722151810666,
 305: 0.45697391565872514,
 316: 0.6328751888246151,
 319: 0.4412302686149321,
 329: 0.35094789574252283,
 344: 0.3486399848523023,
 385: 0.7267501219892796,
 388: 0.3015992875682263,
 398: 0.33561777510291724,
 672: 0.491147165832829,
 742: 0.29980974019676665,
 758: 0.3221957820572615,
 853: 0.3087219610020836,
 927: 0.3648318692331811,
 934: 0.33905094756715126,
 954: 0.40808688788043734}